# Creating an agent with MCP

The [Model Context Protocol](https://www.anthropic.com/news/model-context-protocol) (MCP) introduced by Anthropic has proven to be a popular method for providing an AI agent with access to a variety of tools. [This Huggingface blog post ](https://huggingface.co/blog/Kseniase/mcp) has a nice explanation of MCP.  In this tutorial, we'll build an agent that is able to leverage MCP server provided tools.

Note: because this tutorial relies upon advanced stdio/stderr communication using the MCP Server, it cannot be run on Google Colab.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent'

import nest_asyncio

nest_asyncio.apply()

## Configure the Agent

Now it's time to configure the agent! At this stage you have a few choices:

### Pick the framework

We support a variety of underlying agent frameworks (OpenAI, Smolagents, Langchain, TinyAgent, etc), which all have their own particular agentic AI implementations. For this tutorial's simple use case, any of the frameworks should work just fine, but any-agent makes it easy to try out a different framework later, if we so choose. For this example, we will use the [TinyAgent](frameworks/tinyagent.md) framework.  

### Pick an LLM

Regardless of which agent framework you choose, each framework supports LiteLLM, which is a proxy that allows us to use whichever LLM inside the framework, hosted on by any provider. For example, we could use a local model via llama.cpp or llamafile, a google hosted gemini model, or a AWS bedrock hosted Llama model. For this example, let's use Mistral AI's mistral/mistral-small-latest.

### Pick which tools to use

 In this example, we'll add a few MCP servers that we host locally, which means we'll use a Stdio MCP server. If an MCP Server is already running and hosted elsewhere, you can use an SSE connection to access it. You can browse some of the officially supported MCP servers [here](https://github.com/modelcontextprotocol/servers/tree/main?tab=readme-ov-file).

 Lets give use two MCP servers: 
 
 * [Time](https://github.com/modelcontextprotocol/servers/tree/main/src/time): so the agent can know what time/day it is.
 * [Airbnb](https://github.com/openbnb-org/mcp-server-airbnb): so the agent can browse airbnb listings

 I will also add a custom send_message tool, that way it can ask us additional questions before getting its final answer!
  

In [2]:
import os
from getpass import getpass

if "MISTRAL_API_KEY" not in os.environ:
    print("MISTRAL_API_KEY not found in environment!")
    api_key = getpass("Please enter your MISTRAL_API_KEY: ")
    os.environ["MISTRAL_API_KEY"] = api_key
    print("MISTRAL_API_KEY set for this session!")
else:
    print("MISTRAL_API_KEY found in environment.")

MISTRAL_API_KEY found in environment.


In [ ]:
from any_agent import AgentConfig, AnyAgent
from any_agent.config import MCPStdio

# This MCP Tool relies upon uvx https://docs.astral.sh/uv/getting-started/installation/
time_tool = MCPStdio(
    command="uvx",
    args=["mcp-server-time", "--local-timezone=America/New_York"],
    tools=[
        "get_current_time",
    ],
    client_session_timeout_seconds=30,
)

# This MCP tool relies upon npx https://docs.npmjs.com/cli/v8/commands/npx which comes standard with npm
airbnb_tool = MCPStdio(
    command="npx",
    args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
    client_session_timeout_seconds=30,
)


# This is a custom tool that we will provide to the agent. For the agent to use the tool, we must provide a docstring
# and also have proper python typing for input and output parameters
def send_message(message: str) -> str:
    """Display a message to the user and wait for their response.

    Args:
        message: str
            The message to be displayed to the user.

    Returns:
        str: The response from the user.

    """
    if os.environ.get("IN_PYTEST") == "1":
        return "2 people, next weekend, low budget. Do not ask for any more information or confirmation."
    return input(message + " ")


agent = AnyAgent.create(
    "tinyagent",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(
        model_id="mistral/mistral-small-latest",
        tools=[airbnb_tool, time_tool, send_message],
    ),
)

## Run the Agent

Now we've configured our agent, so it's time to run it! Since it has access to airbnb listings as well as the current time, it's a perfect fit for helping me find a nice airbnb for the weekend.


In [4]:
prompt = """
I am looking to book an airbnb next weekend near Ohiopyle, PA. Can you help me plan this? Figure out the time, then ask me some questions and lets figure this out together. Once I tell you what you need to know, find and return some options for me.
"""

agent_trace = agent.run(prompt)

╭──────────────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "You are an agent - please keep going until the user's query is completely resolved, before  │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "\nI am looking to book an airbnb next weekend near Ohiopyle, PA. Can you help me plan this? │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_current_time",                                                                        │ │
│ │     "tool.args": "{\"timezone\":\"America/New_York\"}"                                                      │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 641,                                                                                      │ │
│ │   "output_tokens": 18,                                                                                      │ │
│ │   "input_cost": 0.0002564,                                                                                  │ │
│ │   "output_cost": 2.88e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── EXECUTE_TOOL: get_current_time ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "timezone": "America/New_York"                                                                            │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Output ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "meta=None content=[TextContent(type='text', text='{\\n  \"timezone\": \"America/New_York\",\\n  \"datetime │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "send_message",                                                                            │ │
│ │     "tool.args": "{\"message\":\"Today is May 23, 2025, and you are looking to book an Airbnb next weekend  │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 732,                                                                                      │ │
│ │   "output_tokens": 194,                                                                                     │ │
│ │   "input_cost": 0.00029279999999999996,                                                                     │ │
│ │   "output_cost": 0.0003104                                                                                  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: send_message ───────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "message": "Today is May 23, 2025, and you are looking to book an Airbnb next weekend near Ohiopyle, PA.  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Output ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "Let's do Friday through Sunday, 2 adults, whole home, maybe some place near the woods! No specific require │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "airbnb_search",                                                                           │ │
│ │     "tool.args": "{\"location\":\"Ohiopyle, PA\",\"checkin\":\"2025-05-30\",\"checkout\":\"2025-06-01\",\"a │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 968,                                                                                      │ │
│ │   "output_tokens": 49,                                                                                      │ │
│ │   "input_cost": 0.0003872,                                                                                  │ │
│ │   "output_cost": 7.84e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: airbnb_search ──────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "location": "Ohiopyle, PA",                                                                               │ │
│ │   "checkin": "2025-05-30",                                                                                  │ │
│ │   "checkout": "2025-06-01",                                                                                 │ │
│ │   "adults": 2,                                                                                              │ │
│ │   "minPrice": 0                                                                                             │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Output ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "meta=None content=[TextContent(type='text', text='{\\n  \"searchUrl\": \"https://www.airbnb.com/s/Ohiopyle │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "airbnb_listing_details",                                                                  │ │
│ │     "tool.args": "{\"id\":\"53007015\",\"checkin\":\"2025-05-30\",\"checkout\":\"2025-06-01\",\"adults\":2} │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 7887,                                                                                     │ │
│ │   "output_tokens": 42,                                                                                      │ │
│ │   "input_cost": 0.0031547999999999997,                                                                      │ │
│ │   "output_cost": 6.72e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── EXECUTE_TOOL: airbnb_listing_details ──────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "id": "53007015",                                                                                         │ │
│ │   "checkin": "2025-05-30",                                                                                  │ │
│ │   "checkout": "2025-06-01",                                                                                 │ │
│ │   "adults": 2                                                                                               │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Output ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "meta=None content=[TextContent(type='text', text='{\\n  \"listingUrl\": \"https://www.airbnb.com/rooms/530 │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: mistral/mistral-small-latest ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "airbnb_listing_details",                                                                  │ │
│ │     "tool.args": "{\"id\":\"52684053\",\"checkin\":\"2025-05-30\",\"checkout\":\"2025-06-01\",\"adults\":2} │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 8872,                                                                                     │ │
│ │   "output_tokens": 42,                                                                                      │ │
│ │   "input_cost": 0.0035488,                                                                                  │ │
│ │   "output_cost": 6.72e-05                                                                                   │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── EXECUTE_TOOL: airbnb_listing_details ──────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "id": "52684053",                                                                                         │ │
│ │   "checkin": "2025-05-30",                                                                                  │ │
│ │   "checkout": "2025-06-01",                                                                                 │ │
│ │   "adults": 2                                                                                               │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Output ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "meta=None content=[TextContent(type='text', text='{\\n  \"listingUrl\": \"https://www.airbnb.com/rooms/526 │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## View the results 

The `agent.run` method returns an AgentTrace object, which has a few convenient attributes for displaying some interesting information about the run.

In [7]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Total Tokens: {agent_trace.tokens.total_tokens:,}")
print(f"Total Cost (USD): {agent_trace.cost.total_cost:.6f}")

I found some Airbnb options near Ohiopyle, PA for next weekend (May 30 to June 1) for 2 adults looking for a whole home, preferably near the woods and ideally with a hot tub. Here are a couple of highlighted listings:

1. Yoder School Guest House (Grantsville, MD)  
- Private hot tub  
- Entire home  
- Good for outdoor activities (mountain biking, hiking, rafting)  
- $297 for 2 nights  
- [Link to listing](https://www.airbnb.com/rooms/53007015?check_in=2025-05-30&check_out=2025-06-01&adults=2)

2. Flanigan Farmhouse (Confluence, PA)  
- Cozy modern 3-bedroom farmhouse on 4 acres  
- Mountain views, private patio, fire pit, hammock  
- $356 for 2 nights  
- [Link to listing](https://www.airbnb.com/rooms/52684053?check_in=2025-05-30&check_out=2025-06-01&adults=2)

Would you like me to show more listings or focus on any particular features?
Duration: 72.20 seconds
Usage: 50577
Total Cost (USD): 0.021473
